<a href="https://colab.research.google.com/github/prtk2403/kristalbal_assignment/blob/main/Kristalbal_AI_Assignment_Pratheek_Nistala.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Analysis performed by - Pratheek Nistala (21BDS0181)

---
Contact Information

[Github](https://github.com/prtk2403) <br/>
[X](https://x.com/xyzprtk)

**Note**

This analysis was performed on Sunday, May 25, 2024, at 11:20 AM. While the time and location (Hyderabad, Telangana, India) are noted, they do not have a material impact on the analysis of the historical dataset provided.

### **1\. The Business Problem**

A hotel chain is facing a significant challenge in managing the inventory for its bars across multiple locations. The core problem is a recurring pattern of **stockouts** on high-demand items and **overstocking** of slow-moving items. These issues lead to:

*   **Increased Operational Costs**: Overstocking ties up capital in unsold inventory, leading to storage costs and potential spoilage. Stockouts of popular items can lead to lost sales opportunities.
    
*   **Decreased Guest Satisfaction**: When guests cannot order their preferred drinks, it leads to a negative experience, potentially impacting their overall perception of the hotel and their loyalty.
    

This project aims to develop a data-driven system to forecast demand for each beverage and recommend optimal inventory levels to be maintained at each bar. This will help the hotel chain to:

*   **Minimize Stockouts**: Ensure that popular items are always available, maximizing revenue and guest satisfaction.
    
*   **Reduce Overstocking**: Free up capital and reduce waste by holding less of the slow-moving inventory.
    
*   **Automate and Standardize**: Provide a consistent and automated way for bar managers to make inventory decisions, reducing the reliance on guesswork and manual processes.


**2\. Data Exploration and Preprocessing**

First, we load the dataset and perform some initial exploration to understand its structure and content. The dataset contains records of alcohol consumption from different bars, including the date and time of service, bar name, alcohol type, brand, and consumption data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
from google.colab import files
import io
print("Please upload your dataset.")
uploaded = files.upload()

# Check if the file was uploaded and get its name
filename = next(iter(uploaded))
print(f"\nReading the uploaded file: {filename}")
df = pd.read_csv(io.BytesIO(uploaded[filename]))

print("\nFirst 5 rows of the dataset:")
print(df.head())


print("\nDataset information:")
df.info()

print("\nMissing values count:")
print(df.isnull().sum())

Please upload your dataset.


Saving Consumption Dataset - Dataset.csv to Consumption Dataset - Dataset (1).csv

Reading the uploaded file: Consumption Dataset - Dataset (1).csv

First 5 rows of the dataset:
  Date Time Served       Bar Name Alcohol Type      Brand Name  \
0   1/1/2023 19:35    Smith's Bar          Rum  Captain Morgan   
1   1/1/2023 10:07    Smith's Bar         Wine     Yellow Tail   
2   1/1/2023 11:26  Johnson's Bar        Vodka      Grey Goose   
3   1/1/2023 13:53  Johnson's Bar         Beer           Coors   
4   1/1/2023 22:28  Johnson's Bar         Wine     Yellow Tail   

   Opening Balance (ml)  Purchase (ml)  Consumed (ml)  Closing Balance (ml)  
0               2555.04        1824.84            0.0               4379.88  
1               1344.37           0.00            0.0               1344.37  
2               1034.28           0.00            0.0               1034.28  
3               2194.53           0.00            0.0               2194.53  
4               1020.90           0

The data needs to be preprocessed to make it suitable for time-series forecasting. We will:

1.  Convert the 'Date Time Served' column to a datetime object and set it as the index.
    
2.  Create a unique identifier for each item at each bar by combining 'Bar Name', 'Alcohol Type', and 'Brand Name'.
    
3.  Aggregate the consumption data by day for each unique item.
    

In [ ]:
# Convert 'Date Time Served' to datetime and set as index
df['Date Time Served'] = pd.to_datetime(df['Date Time Served'])
df.set_index('Date Time Served', inplace=True)

# Create a unique item identifier
df['Item'] = df['Bar Name'] + ' - ' + df['Alcohol Type'] + ' - ' + df['Brand Name']

# Aggregate consumption by day for each item
daily_consumption = df.groupby(['Item', pd.Grouper(freq='D')])['Consumed (ml)'].sum().reset_index()
daily_consumption.rename(columns={'Date Time Served': 'Date'}, inplace=True)

# Display the aggregated data
print(daily_consumption.head())

                                Item       Date  Consumed (ml)
0  Anderson's Bar - Beer - Budweiser 2023-01-09         159.07
1  Anderson's Bar - Beer - Budweiser 2023-01-18         238.21
2  Anderson's Bar - Beer - Budweiser 2023-01-21         303.50
3  Anderson's Bar - Beer - Budweiser 2023-01-29         238.29
4  Anderson's Bar - Beer - Budweiser 2023-02-02         225.56


**3\. Assumptions**

In any real-world modeling project, we need to make some assumptions. Here are the key assumptions for this project:

*   **Lead Time**: We assume a constant lead time of **3 days** for all items. This is the time it takes from placing an order to receiving the delivery. In a real-world scenario, this could be more dynamic.
    
*   **Service Level**: We are aiming for a **95% service level**, which means we want to be in-stock 95% of the time. This corresponds to a Z-score of approximately 1.65.
    
*   **Review Period**: We assume that the inventory is reviewed and orders are placed **daily**.

**4\. Demand Forecasting**

We will forecast the demand for each item at each bar. We'll start with a simple baseline model and then use a more sophisticated forecasting model.

#### **4.1. Baseline Model: Simple Moving Average (SMA)**

A simple moving average is a good baseline to see how a simple model performs. We will use a 7-day moving average to predict the next day's demand.

In [ ]:
def sma_forecast(series, window):
    """Calculates the simple moving average forecast."""
    return series.rolling(window).mean().iloc[-1]

# For Example
item_data = daily_consumption[daily_consumption['Item'] == "Smith's Bar - Rum - Captain Morgan"].set_index('Date')['Consumed (ml)']
sma_pred = sma_forecast(item_data, 7)
print(f"SMA Forecast for next day: {sma_pred:.2f} ml")

SMA Forecast for next day: 277.14 ml


**4.2. Exponential Smoothing Model**

Exponential smoothing is a robust forecasting method that is easy to implement and works well for many time series. It's a good choice for this problem because it can capture trends and seasonality in the data. We will use the Holt-Winters method, which can model both trend and seasonality.

In [ ]:
# Select the data for a single item
item_data_raw = daily_consumption[daily_consumption['Item'] == "Smith's Bar - Rum - Captain Morgan"].set_index('Date')['Consumed (ml)']
item_data = item_data_raw.asfreq('D').fillna(0)

def exponential_smoothing_forecast(series, trend='add', seasonal='add', seasonal_periods=7):
    """Calculates the exponential smoothing forecast."""
    fit = ExponentialSmoothing(
        series,
        seasonal_periods=seasonal_periods,
        trend=trend,
        seasonal=seasonal,
        initialization_method="estimated",
    ).fit()
    return fit.forecast(1).iloc[0]

# Example
es_pred = exponential_smoothing_forecast(item_data)
print(f"Exponential Smoothing Forecast for next day: {es_pred:.2f} ml")

Exponential Smoothing Forecast for next day: 56.91 ml


**5\. Inventory Recommendation System**

Now that we have a demand forecast, we can build the inventory recommendation system. The system will calculate the following for each item:

*   **Safety Stock**: Extra stock to buffer against forecast inaccuracies and demand variability.
    
    *   Safety Stock = Z-score \* Standard Deviation of Lead Time Demand
        
*   **Reorder Point (ROP)**: The inventory level at which a new order should be placed.
    
    *   Reorder Point = (Average Daily Usage \* Lead Time) + Safety Stock
        
*   **Par Level (Order-up-to-Level)**: The maximum desired inventory level.
    
    *   Par Level = Reorder Point + Average consumption during the review period
        

In [ ]:
def calculate_inventory_levels(series, lead_time=3, service_level_z=1.65):
    """Calculates safety stock, reorder point, and par level."""
    avg_daily_usage = series.mean()
    std_daily_usage = series.std()

    # Calculate standard deviation of lead time demand
    std_lead_time_demand = std_daily_usage * np.sqrt(lead_time)

    # Calculate safety stock
    safety_stock = service_level_z * std_lead_time_demand

    # Calculate reorder point
    reorder_point = (avg_daily_usage * lead_time) + safety_stock

    # Calculate par level
    par_level = reorder_point + avg_daily_usage

    return {
        'Safety Stock (ml)': safety_stock,
        'Reorder Point (ml)': reorder_point,
        'Par Level (ml)': par_level
    }

# For Example
inventory_levels = calculate_inventory_levels(item_data)
print(inventory_levels)

{'Safety Stock (ml)': np.float64(403.47269796122043), 'Reorder Point (ml)': np.float64(584.7785176333516), 'Par Level (ml)': np.float64(645.2137908573953)}


**6\. Simulation**

To understand how our system would perform in practice, we will run a simulation. The simulation will track inventory levels over time, using our forecasting and inventory recommendation logic. We will track key performance indicators (KPIs) like stockouts and service level.

In [ ]:
def simulate_inventory(series, lead_time=3, service_level_z=1.65):
    """Simulates inventory management over a period."""
    inventory_levels = calculate_inventory_levels(series, lead_time, service_level_z)
    par_level = inventory_levels['Par Level (ml)']

    inventory = par_level
    stockout_days = 0

    for demand in series:
        if inventory < demand:
            stockout_days += 1
            inventory = 0
        else:
            inventory -= demand

        # Replenish inventory to par level at the end of the day
        inventory = par_level

    service_level = (len(series) - stockout_days) / len(series) * 100
    return {
        'Total Days': len(series),
        'Stockout Days': stockout_days,
        'Service Level (%)': service_level
    }

# For Example
simulation_results = simulate_inventory(item_data)
print(simulation_results)

{'Total Days': 366, 'Stockout Days': 0, 'Service Level (%)': 100.0}


**7\. Report and Conclusion**

#### **7.1. How the System Performs and Potential Improvements**

The system's performance, based on the simulation, is quite good. For the example item, we achieved a high service level, which means we were able to meet customer demand most of the time.

**Improvements**:

*   **Dynamic Lead Times**: Incorporate dynamic lead time forecasting, as supplier lead times can vary.
    
*   **Promotions and Events**: The model can be improved by including a feature that accounts for promotions, holidays, and local events, which can significantly impact demand.
    
*   **More Sophisticated Models**: For items with complex demand patterns, more advanced models like SARIMA or machine learning models (e.g., Gradient Boosting, LSTMs) could be explored.
    

#### **7.2. Real-World Implementation**

To implement this solution in a real hotel, the following steps would be necessary:

1.  **Data Integration**: Set up an automated pipeline to collect and process consumption data from the hotel's Point of Sale (POS) system in near real-time.
    
2.  **Dashboarding**: Create a user-friendly dashboard for bar managers that displays the demand forecasts, recommended inventory levels, and alerts for items that are running low.
    
3.  **Training**: Train bar managers on how to use the system and interpret the recommendations.
    
4.  **Monitoring and Retraining**: Continuously monitor the model's performance and retrain it periodically with new data to ensure it remains accurate.


#### **7.3. (Optional) Scalability and Production Monitoring**

*   **Scalability**: At scale, the system would need to handle data from hundreds of bars and thousands of items. This would require a robust data infrastructure, likely using cloud-based services for data storage (e.g., a data warehouse) and computation (e.g., a distributed computing framework like Spark).
    
*   **Production Monitoring**: In a production environment, we would need to track:
    
    *   **Model Accuracy**: Metrics like Mean Absolute Error (MAE) and Root Mean Squared Error (RMSE) to track how well the forecasts are performing.
        
    *   **Business KPIs**: Track the impact on key business metrics like stockout rates, inventory turnover, and guest satisfaction scores to measure the system's ROI.